---
layout: post
title: C Bounded Model Checker 2: Loops, Contracts, Equivalence checking
---

## Loops

Ok, we've painted a rosy picture so far. Obviously CBMC does not scale to arbitrarily large and difficult problems

The basic technique is to unwind them. All is not lost, many loops (for loops in particular) can be completely unwound. You can add `--unwinding-assertions` to know if you've covered all possible executions. Ever if you can't, passing these chekcs does give you some confidence.

If you're going to the next level there is also the ability to add invariant annotations. I'm not sure if CBMC can infer these successfully

On bigger problems, I've had some success fiddling with the 

https://diffblue.github.io/cbmc/cprover-manual/md_cbmc_tutorial.html

In [ ]:
%%file 


## Contracts
https://diffblue.github.io/cbmc/contracts-user.html

# Comparative Checking
Sum forward is the same as sum backwards

In [24]:
%%file /tmp/sum.c

extern int array[10];
 
int sum()
{
  unsigned i, sum;
 
  sum = 0;
  for(i = 0; i < 10; i++)
    sum += array[i];
 
  return sum;
}

int sum_back()
{
  int i, sum; // Interesting. i had a bug when these were unsgined. 
  // Of course the for loop does something weird in that case
 
  sum = 0;
  for(i = 9; i >= 0; i--)
    sum += array[i];
  return sum;
}

int main(){
  assert(sum() == sum_back());
}

Overwriting /tmp/sum.c


In [28]:
! cbmc /tmp/sum.c --boolector # Sat solver is super slow on this one

CBMC version 5.95.1 (cbmc-5.95.1) 64-bit x86_64 linux
Parsing /tmp/sum.c
Converting
Type-checking sum
file /tmp/sum.c line 26 function main: function 'assert' is not declared
Generating GOTO Program
Adding CPROVER library (x86_64)
Removal of function pointers and virtual functions
Generic Property Instrumentation
Running with 8 object bits, 56 offset bits (default)
Starting Bounded Model Checking
Unwinding loop sum.0 iteration 1 file /tmp/sum.c line 9 function sum thread 0
Unwinding loop sum.0 iteration 2 file /tmp/sum.c line 9 function sum thread 0
Unwinding loop sum.0 iteration 3 file /tmp/sum.c line 9 function sum thread 0
Unwinding loop sum.0 iteration 4 file /tmp/sum.c line 9 function sum thread 0
Unwinding loop sum.0 iteration 5 file /tmp/sum.c line 9 function sum thread 0
Unwinding loop sum.0 iteration 6 file /tmp/sum.c line 9 function sum thread 0
Unwinding loop sum.0 iteration 7 file /tmp/sum.c line 9 function sum thread 0
Unwinding loop sum.0 iteration 8 file /tmp/sum.c line 

A very interesting topic is that of security properties / information leakage. how do you write specs for this? What is a definition of information leakage?

In [29]:
%%file /tmp/leak.c
#include <assert.h>
int safeprog(int low, int high){
    int foo = low ^ high;
    foo = ((foo << 1) ^ high) >> 1;
    return foo;
}

int main(){
    int high = nondet_int();
    int high1 = nondet_int();
    int low = nondet_int();
    //int high, high1, low;

    assert(safeprog(low,high) == safeprog(low,high1));
    return 0;
}

Overwriting /tmp/leak.c


In [30]:
!cbmc /tmp/leak.c

CBMC version 5.95.1 (cbmc-5.95.1) 64-bit x86_64 linux
Parsing /tmp/leak.c
Converting
Type-checking leak
file /tmp/leak.c line 9 function main: function 'nondet_int' is not declared
Generating GOTO Program
Adding CPROVER library (x86_64)
Removal of function pointers and virtual functions
Generic Property Instrumentation
Running with 8 object bits, 56 offset bits (default)
Starting Bounded Model Checking
Runtime Symex: 0.000956779s
size of program expression: 54 steps
simple slicing removed 5 assignments
Generated 1 VCC(s), 1 remaining after simplification
Runtime Postprocess Equation: 1.647e-05s
Passing problem to propositional reduction
converting SSA
Runtime Convert SSA: 0.000395847s
Running propositional reduction
Post-processing
Runtime Post-process: 3.34e-06s
Solving with MiniSAT 2.2.1 with simplifier
703 variables, 921 clauses
SAT checker: instance is SATISFIABLE
Runtime Solver: 0.00037143s
Runtime decision procedure: 0.000804985s

** Results:
/tmp/leak.c function main
[main.asser

# Bits and Bobbles

Here's a list of serious C projects with CBMC instrumentation, many from Amazon: https://model-checking.github.io/cbmc-training/projects.html. They are playing a similar role to unit tests for library functions.

SV Comp  https://sv-comp.sosy-lab.org/ - CPAchecker UAutomizer are very successful in the competition. I have not used them as much

Klee, symcc are symbolic executors. Similar in many respects to a bounded model checker. I think the biggest philosophical difference is they aren't really centered around ensuring bug absence, instead around bug finding, which is a little different.

Frama-C
VST
https://github.com/verifast/verifast graham seemed to like this one


[esbmc](http://esbmc.org/) is a C bounded model checker.

Comparing a C function to it's grammar?

```bash
echo "
#include<stdbool.h>

bool check_balance(char *input){
    int count = 0;
    while(*input != '\0'){
        if(*input == '(') count++;
        if(*input == ')') count--;
        input++;
    }
    return count == 0;
" > /tmp/parens.c
cbmc /tmp/parens.c 
```


On my last post, I got a lot of guff for having undefined behavior. CBMC is not great for detecting this stuff. Ok, so what can I use?

Indeed `-Wall -Wpedantic` notices. Can I trick it?

In [35]:
%%file /tmp/ub.c
#include <assert.h>
int main(){
    int x;
    x++;
    assert(x != 12345);
}

Overwriting /tmp/ub.c


In [38]:
! clang -O1 -Wall -Wpedantic -fsanitize=undefined /tmp/ub.c -o /tmp/ub.out && /tmp/ub.out

/tmp/ub.c:2:9: warning: a function declaration without a prototype is deprecated in all versions of C [-Wstrict-prototypes]
    2 | int main(){
      |         ^
      |          void
/tmp/ub.c:4:5: warning: variable 'x' is uninitialized when used here [-Wuninitialized]
    4 |     x++;
      |     ^
/tmp/ub.c:3:10: note: initialize the variable 'x' to silence this warning
    3 |     int x;
      |          ^
      |           = 0
2 warnings generated.


In [40]:
! objdump -d /tmp/ub.out | grep -A 10 main.: # This is a disassembly of the main function of the compiled program.
 

000000000002ea28 <main>:
   2ea28:	31 c0                	xor    %eax,%eax
   2ea2a:	c3                   	ret    
   2ea2b:	0f 1f 44 00 00       	nopl   0x0(%rax,%rax,1)

000000000002ea30 <atexit>:
   2ea30:	f3 0f 1e fa          	endbr64 
   2ea34:	48 8b 15 3d 57 01 00 	mov    0x1573d(%rip),%rdx        # 44178 <__dso_handle>
   2ea3b:	31 f6                	xor    %esi,%esi
   2ea3d:	e9 ae 66 fd ff       	jmp    50f0 <__cxa_atexit@plt>



There's nothing there. Indeed, the compiler fused it all out.

In [10]:
%%file /tmp/foo.c
int mystrcmp(const char *cs, const char *ct) {
  unsigned char c1, c2;
  while (1) {
    c1 = *(cs++);
    c2 = *(ct++);
    if (c1 != c2)
      return c1 < c2 ? -1 : 1;
    if (!c1)
      break;
  }
  return 0;
}

int foo(char **argv) {
  char *command = argv[1];
  //assert(mystrcmp("STORE", command) != 0);
  assert(strcmp("STORE", command) != 0);
  return 0;
}

Overwriting /tmp/foo.c


In [5]:
!cbmc --help


* *             CBMC 5.95.1 (cbmc-5.95.1) 64-bit            * *
* *                 Copyright (C) 2001-2018                 * *
* *              Daniel Kroening, Edmund Clarke             * *
* * Carnegie Mellon University, Computer Science Department * *
* *                  kroening@kroening.com                  * *
* *        Protected in part by U.S. patent 7,225,417       * *

Usage:                       Purpose:

 cbmc [-?] [-h] [--help]      show this help
 cbmc --version               show version and exit
 cbmc [options] file.c ...    perform bounded model checking

Analysis options:
 --show-properties            show the properties, but don't run analysis
 --symex-coverage-report f    generate a Cobertura XML coverage report in f
 --property id                only check one specific property
 --trace                      give a counterexample trace for failed properties
 --stop-on-fail               stop analysis once a failed property is detected 
                         

In [14]:
!cbmc --function foo /tmp/foo.c --trace

CBMC version 5.95.1 (cbmc-5.95.1) 64-bit x86_64 linux
Parsing /tmp/foo.c
Converting
Type-checking foo
file /tmp/foo.c line 17 function foo: function 'strcmp' is not declared
file /tmp/foo.c line 17 function foo: function 'assert' is not declared
Generating GOTO Program
Adding CPROVER library (x86_64)
file <builtin-library-strcmp> line 9: warning: implicit function declaration "strcmp"
old definition in module foo file /tmp/foo.c line 17 function foo
signed int (void)
new definition in module <built-in-library> file <builtin-library-strcmp> line 9
signed int (const char *s1, const char *s2)
Removal of function pointers and virtual functions
Generic Property Instrumentation
Running with 8 object bits, 56 offset bits (default)
Starting Bounded Model Checking
Unwinding loop strcmp.0 iteration 1 file <builtin-library-strcmp> line 44 function strcmp thread 0
Unwinding loop strcmp.0 iteration 2 file <builtin-library-strcmp> line 44 function strcmp thread 0
Unwinding loop strcmp.0 iteration 3 

In [9]:
!cbmc /tmp/foo.c --show-goto-functions

CBMC version 5.95.1 (cbmc-5.95.1) 64-bit x86_64 linux
Parsing /tmp/foo.c
Converting
Type-checking foo
file /tmp/foo.c line 16 function foo: function 'assert' is not declared
Generating GOTO Program
Adding CPROVER library (x86_64)
Removal of function pointers and virtual functions
Generic Property Instrumentation
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

foo /* foo */
        // 29 file /tmp/foo.c line 15 function foo
        DECL foo::1::command : signedbv[8]*
        // 30 file /tmp/foo.c line 15 function foo
        ASSIGN foo::1::command := *(foo::argv + cast(1, signedbv[64]))
        // 31 file /tmp/foo.c line 16 function foo
        DECL foo::$tmp::return_value_mystrcmp : signedbv[32]
        // 32 file /tmp/foo.c line 16 function foo
        CALL foo::$tmp::return_value_mystrcmp := mystrcmp(address_of("STORE"[0]), foo::1::command)
        // 33 file /tmp/foo.c line 16 function foo
        ASSERT foo::$tmp::return_value_mystrcmp ≠ 0 // assertion return_value_mystrcmp != 0
    